# Ylioppilastutkintodata

Tämä notebook käyttää Vipunen APIn kautta ladattua ylioppilastutkintadataa. Notebookin ohjeet ovat suomeksi, koska datan otsakkeet ovat suomeksi ja käytännössä dataa on vaikea käpistellä ymmärtämättä suomea. Koodi ja kommentit sentään ovat englanniksi.

Aloita ajamalla allaoleva koodinpätkä, joka lataa datan. Koodin voit ajamalla valitsemalla solun ja painamalla ctrl+R tai klikkaamalla ylärivin play-nappia.

In [ ]:
# download the data to a local file as it is almost 100 megabytes
# this will take a while
from urllib.request import urlretrieve

destination = "/tmp/ytl_arvosanat.json"
url = "http://api.vipunen.fi/api/resources/ytl_arvosanat/data"
urlretrieve(url, destination)

Lataus on valmis kun harmaan laatikon alle ilmestyy rivi, jonka vasemmassa reunassa on teksti joka näyttää osapuilleen seuraavalta.
```
Out[5]: ('/tmp/ytl_arvosanat.json', <http.client.HTTPMessage at 0x7f81c437b320>)
```

Kun data on latautunut, voit ajaa seuraavan solun.

In [ ]:
import pandas as pd
import json
with open(destination, "r") as file_:
    df = pd.DataFrame.from_records(json.load(file_))
    
df["arvosanapisteet"] = df[["arvosanapisteet"]].astype(int)


# pandas is not good about assuming which variables are categorical so we need to help it a bit
categorical_variables = ["opiskelijaKoodi", "arvosana", 
                         "arvosanaKoodi", "koe", "koeKoodi", "lukio", "lukioKoodi", 
                         "opetuskieli", "opetuskieliKoodi", "sukupuoli", "sukupuoliKoodi", 
                         "oppilaitostyyppi", "oppilaitostyyppiKoodi", "tutkintokerta", 
                         "tutkintokertaKoodi"]
for var in categorical_variables:
    df[var] = df[var].astype('category')

original_df = df

Noin, nyt data on oikeastaan aika siistissä muodossa ja sitä kelpaa käpistellä esimerkiksi [pandas-kirjastolla](http://pandas.pydata.org/).

Rajoitetaan vielä itseämme hieman ja suodatetaan kaikesta datasta vain kevään 2017 kirjoitukset. Datasetistä löytyvät myös 2015S, 2016K ja 2016S mikäli haluat tutkia lisää.

In [ ]:
df = original_df[original_df.tutkintokertaKoodi == "2017K"]


In [ ]:
df.arvosanapisteet.describe()

## Englanti, pitkä oppimäärä

Tarkastellaan ensin opetuskielen ja pitkän englannin suhdetta.

In [ ]:
df[df.koe == "Englanti, pitkä oppimäärä"].hist(bins=range(9), by="opetuskieli", sharex=True)

Jakauma vaihtelee erittäin paljon tutkintkerrasta toiseen, erityisesti ruotsinkielisillä koska volyymi on niin pientä.

Suhteuttaminen ilmiöiden kokoon on paikallaan.

In [ ]:
df[df.koe == "Englanti, pitkä oppimäärä"].hist(bins=range(9), by="opetuskieli", sharex=True, sharey=True)


## Äidinkieli

Tästä klassisesta esimerkistä ei tarvitse varmaan selittää.

In [ ]:
# Z A and O are codes for Essay writing
df[df["koeKoodi"].isin(["Z", "A", "O"])].hist(bins=range(9), by="sukupuoli", sharex=True, sharey=True)

## Matematiikka

Pitkä matematiikka on koekoodilla M ja lyhyt koodilla N.

In [ ]:
df[df.koeKoodi == "M"].hist(bins=range(9), by="sukupuoli", sharey=True)

On "yleisesti tiedossa", että pitkä matematiikka on nuorten miesten laji.


In [ ]:
df[df.koeKoodi == "N"].hist(bins=range(9), by="sukupuoli", sharey=True)

Lyhyen matematiikan kohdalla taas hyvin kirjoittavien miesten osuus on pienempi.

In [ ]:
df[df["koeKoodi"].isin(["M", "N"])].hist(bins=range(9), by="sukupuoli", sharey=True)

Arvosanajakauma on paljon tasaisempi kun lyhyen ja pitkän matematiikan kirjoittajat plotataan yhdessä. Käytössä olevan datan perusteella ei voida arvioida johtuuko ero laadullisesta erosta sukupuolten välillä vai erilaisista ympäristötekijöistä.